### csv파일 수정하여 고객정보 수정하기   
몸무게/연락처/운동량 정도는 수정가능해야 함   
몸무게는 기초대사율에 영향을 주기 때문에 주기적으로 측정하여 수정해줘야하고 
운동량은 활동대사량을 결정할때 사용한다

In [ ]:
import pandas as pd

def update_customer_weight(csv_path, customer_id, new_weight):
    """
    특정 고객의 몸무게를 수정하는 함수.
    
    Parameters:
        csv_path (str): 고객 정보가 저장된 CSV 파일 경로.
        customer_id (str): 수정할 고객의 ID.
        new_weight (float): 새 몸무게 값.
    """
    try:
        # CSV 파일 읽기
        customer_data = pd.read_csv(csv_path)

        # 고객 ID로 행 필터링
        if '고객아이디' not in customer_data.columns:
            print("CSV 파일에 '고객아이디' 열이 없습니다.")
            return

        # 고객 정보 찾기
        customer_index = customer_data[customer_data['고객아이디'] == customer_id].index

        if not customer_index.empty:
            # 몸무게 수정
            customer_data.loc[customer_index, '몸무게'] = new_weight
            print(f"고객 ID {customer_id}의 몸무게가 {new_weight}kg으로 수정되었습니다.")
        else:
            print(f"고객 ID {customer_id}에 해당하는 정보를 찾을 수 없습니다.")
            return

        # CSV 파일에 수정된 데이터 저장
        customer_data.to_csv(csv_path, index=False)
        print(f"CSV 파일이 성공적으로 업데이트되었습니다: {csv_path}")

    except FileNotFoundError:
        print(f"CSV 파일 {csv_path}을(를) 찾을 수 없습니다.")
    except Exception as e:
        print(f"오류 발생: {e}")

# 예제 실행
csv_path = "FOOD_DB/food_project_user_info.csv"  # 고객 정보가 저장된 CSV 파일 경로
customer_id = input() # 수정할 고객 ID
new_weight = input()  # 새 몸무게 값

update_customer_weight(csv_path, customer_id, new_weight)


### 고객정보 수정페이지 streamlit 구현 및 정보수정

In [ ]:
import streamlit as st
import pandas as pd

# CSV 파일 경로
CSV_PATH = "customer_data.csv"

def load_data(file_path):
    """CSV 파일 읽기"""
    try:
        data = pd.read_csv(file_path)
        return data
    except FileNotFoundError:
        st.error(f"CSV 파일 '{file_path}'이(가) 존재하지 않습니다.")
        return pd.DataFrame()

def save_data(data, file_path):
    """수정된 데이터를 CSV 파일에 저장"""
    try:
        data.to_csv(file_path, index=False)
        st.success("CSV 파일이 성공적으로 업데이트되었습니다!")
    except Exception as e:
        st.error(f"데이터 저장 중 오류 발생: {e}")

def get_customer_info(data, customer_id):
    """특정 고객의 정보를 가져옴"""
    customer_row = data[data['customer_id'] == customer_id]
    if not customer_row.empty:
        return customer_row.iloc[0]
    return None

# Streamlit 앱 구성
st.title("고객 정보 수정 페이지")

# CSV 데이터 로드
customer_data = load_data(CSV_PATH)

if not customer_data.empty:
    # 고객 ID 입력
    customer_id = st.text_input("수정할 고객 ID를 입력하세요:", "")

    if customer_id:
        # 고객 정보 검색
        customer_info = get_customer_info(customer_data, customer_id)

        if customer_info is not None:
            st.write("### 고객 정보")
            st.write(f"**이름:** {customer_info['name']}")
            st.write(f"**현재 몸무게:** {customer_info['weight']} kg")
            st.write(f"**연락처:** {customer_info['phone']}")

            # 수정 가능한 입력 필드
            new_weight = st.number_input("새 몸무게 (kg)", value=float(customer_info['weight']))
            new_phone = st.text_input("새 연락처", value=customer_info['phone'])

            # 저장 버튼
            if st.button("저장"):
                # 데이터 수정
                customer_data.loc[customer_data['customer_id'] == customer_id, 'weight'] = new_weight
                customer_data.loc[customer_data['customer_id'] == customer_id, 'phone'] = new_phone

                # 수정된 데이터 저장
                save_data(customer_data, CSV_PATH)
        else:
            st.warning(f"고객 ID '{customer_id}'에 해당하는 정보를 찾을 수 없습니다.")
